In [ ]:
# Importiamo le librerie necessarie
from idaes.core import FlowsheetBlock
from pyomo.environ import ConcreteModel, SolverFactory, Var, Constraint, Param, NonNegativeReals
from pyomo.environ import Objective, minimize  # Importiamo anche minimize per l'obiettivo


In [ ]:

# Creiamo il modello IDAES
model = ConcreteModel()

# Creiamo un flusso di processo (flowsheet) nel modello senza il parametro default
model.fs = FlowsheetBlock()

# Definiamo i parametri del sistema (esempio di capacità termica e flusso di massa)
model.fs.heat_capacity = Param(initialize=4.18)  # capacità termica in kJ/(kg*K)
model.fs.mass_flow_rate = Param(initialize=1.0)   # portata di massa in kg/s

# Definiamo variabili per la temperatura di ingresso e uscita
model.fs.inlet_temperature = Var(within=NonNegativeReals, initialize=300)  # temperatura ingresso (K)
model.fs.outlet_temperature = Var(within=NonNegativeReals, initialize=350)  # temperatura uscita (K)

# Equazione di bilancio energetico per lo scambiatore di calore
def energy_balance_eqn(model):
    return (model.fs.heat_capacity * model.fs.mass_flow_rate *
            (model.fs.outlet_temperature - model.fs.inlet_temperature)) == 0


In [ ]:

model.energy_balance = Constraint(rule=energy_balance_eqn)

# Definiamo un obiettivo per minimizzare il consumo energetico
model.obj = Objective(expr=model.fs.heat_capacity * model.fs.mass_flow_rate *
                      (model.fs.outlet_temperature - model.fs.inlet_temperature), sense=minimize)

# Risolviamo il modello
solver = SolverFactory('ipopt')  # Puoi usare 'glpk' o 'ipopt' come solver
solver.solve(model, tee=True)

# Visualizziamo i risultati
print("Temperatura di ingresso:", model.fs.inlet_temperature())
print("Temperatura di uscita:", model.fs.outlet_temperature())
